In [1]:
import os
import cv2
import argparse
import numpy as np

frame_name = "in" + str(1).zfill(6) + ".jpg"
frame = cv2.imread("baseline/input/" + frame_name)

In [2]:
file_handle = open('illumination/eval_frames.txt', 'r')
lines_list = file_handle.readlines()
eval_start, eval_end = (int(val) for val in lines_list[0].split())
file_handle.close()


In [29]:
#Creating background model using training
def intensity_vec(frame):
    return 0.0722 * frame[:, :, 0] + 0.7152 * frame[:, :, 1] + 0.2126 * frame[:, :, 2]


img_list_store = []
for i in range(1, eval_start):
    frame_name = "in" + str(i).zfill(6) + ".jpg"
    frame = cv2.imread('illumination/input/'+frame_name)
    h,w,c = frame.shape
    img_list_store.append(intensity_vec(frame))

median_store = np.full((h,w),0)
std_dev = np.full((h,w),0)
no_images = len(img_list_store)
for i in range(0,h):
    for j in range(0,w):
        intensity_val = []
        for k in range(0,no_images):
            intensity_val.append(img_list_store[k][i,j])
        intensity_val = np.array(intensity_val)
        median_store[i,j] = np.median(intensity_val)
        if median_store[i,j] == 0 :
             median_store[i,j] = 1
        std_dev[i,j] = np.std(intensity_val)
        if std_dev[i,j]== 0:
            std_dev[i,j] = 1

In [30]:
arr1 = np.unique(std_dev)
map_freq = {}
freq_store = []
for i in range(0,len(arr1)):
    freq_store.append(0)
    map_freq[arr1[i]] = i
for i in range(0,h):
    for j in range(0,w):
        freq_store[map_freq[std_dev[i,j]]] += 1

sigma_1 = None
for i in range(1,len(arr1)-1):
    if freq_store[i] > freq_store[i-1] and freq_store[i]> freq_store[i+1]:
        sigma_1 = arr1[i]
        break
print(sigma_1)

3


In [31]:
from scipy import signal

kernel = 1/16* np.array([[1,2,1],[2,4,2],[1,2,1]])
frame_name = "in" + str(eval_start-1).zfill(6) + ".jpg"
frame = cv2.imread('illumination/input/'+frame_name)
old_dt = signal.convolve(np.abs(intensity_vec(frame) - median_store),kernel)



In [43]:
is_shadow_illumination = []
for i in range(eval_start,eval_end+1):
    frame_name = "in" + str(i).zfill(6) + ".jpg"
    frame = cv2.imread('illumination/input/'+frame_name)
    new_dt = signal.convolve(np.abs(intensity_vec(frame) - median_store),kernel)
    store_map = intensity_vec(frame)
    fin_dt = (old_dt + new_dt)/2
    old_dt = new_dt
    
    segmentation_map = np.full((h,w),0)
    is_shadow = np.full((h,w),0)
    count = 0
    for x in range(0,h):
        for y in range(0,w):
            if fin_dt[x,y] > 6*sigma_1 :
                segmentation_map[x,y] = 255
                count +=1
    
    #For shadows and illumination handling
    for x in range(1,h-1):
        for y in range(1,w-1):
            if segmentation_map[x,y] == 0 :
                continue
            arr_store = store_map[x-1:x+2,y-1:y+2]/median_store[x-1:x+2,y-1:y+2]
            std_dev_1 = np.std(arr_store.flatten())
            checkl = True
            checkh = True
            
            for a in range(0,3):
                for b in range(0,3):
                    if arr_store[a,b] > 1 or  arr_store[a,b] < 0.5 :
                        checkl = False
                    if arr_store[a,b] < 1 or  arr_store[a,b] > 1.3 :
                        checkh = False
            if checkl or checkh:
                if std_dev_1 < 0.05 :
                    segmentation_map[x,y] = 0
                    is_shadow[x,y] = 1
    is_shadow_illumination.append(is_shadow)
    
    if(len(is_shadow_illumination) >= 30):
        for x in range(0,h):
            for y in range(0,w):
                tot = 0
                for z in range(1,31):
                    tot += is_shadow_illumination[len(is_shadow_illumination)-z][x,y]
                if tot >= 20:
                    segmentation_map[x,y] = 0
    
    
    pred_name = "gt" + str(i).zfill(6) + ".png"
    if not cv2.imwrite('illumination/output/' + pred_name, segmentation_map):
        print("Save failed")
   
    if i%20 == 0:
        print(i)
                
                        
            
            

60
80
100
120
140
160
180
200
220
240
260
280
300


In [44]:
img = cv2.imread('illumination/groundtruth/'+"gt" + str(1).zfill(6) + ".png")
print(img.shape)

(240, 320, 3)
